### Spencer Investment Strategy Analysis
Objective: run a simulation to measure the returns of investing in the top 100 market cap stocks, but only purchasing after they have dropped to x% below their 52 week high. And then selling those stocks once they surpass y% of my purchase price. Run simulation for z years, re-evaluating stocks every certain number of days. Compare to S&P 500 returns.

Constraints: buy a specific amount of stock every time a buy signal is triggered. Allow the purchase of fractional shares. There is no limit to the amount of money that may be invested at any given time.


### User Inputs

In [1]:
# Buy and sell percentages
buy_after_52_week_high_percentage_drop = 0.2
sell_after_percentage_rise_from_purchase_price = 0.1

# Number of years to look back
lookback_years = 5

# How many days between each buy/sell check
# 1 means buy/sell every day for lookback_years
buy_sell_interval_in_days = 1

# How much money to invest when a buy signal is triggered (in USD)
investment_amount_per_buy_signal = 1000

# Can the same stock be bought again if it is already in the portfolio?
allow_buy_stock_already_in_portfolio = False

# Logging parameters
output_portfolio = False
output_transactions = True
output_outstanding_investment = False

### Installs

In [2]:
!pip install -q yfinance
!pip install -q scipy

### Imports

In [3]:
import yfinance as yf
import scipy.optimize as optimize
import pandas as pd
from datetime import datetime, timedelta

In [4]:
# Calculate the start date. End date of the simulation is 2 days ago. 
start_date = (datetime.now() - timedelta(days=365 * lookback_years + 2)).strftime('%Y-%m-%d')
end_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')

In [5]:
# Hard-coded list of stocks in the S&P 100 as of September 18, 2023
# TODO: Automatically fetch this list for each day of the simulation
# sp100_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN'] # Smaller list, for testing purposes
sp100_stocks = ['AAPL', 'ABBV ', 'ABT ', 'ACN ', 'ADBE ', 'AIG ', 'AMD ', 'AMGN ', 'AMT ', 'AMZN ', 'AVGO ', 'AXP ', 'BA ', 'BAC ', 'BK ', 'BKNG ', 'BLK ', 'BMY ', 'BRK-B ', 'C ', 'CAT ', 'CHTR ', 'CL ', 'CMCSA ', 'COF ', 'COP ', 'COST ', 'CRM ', 'CSCO ', 'CVS ', 'CVX ', 'DE ', 'DHR ', 'DIS ', 'DOW ', 'DUK ', 'EMR ', 'EXC ', 'F ', 'FDX ', 'GD ', 'GE ', 'GILD ', 'GM ', 'GOOG ', 'GOOGL ', 'GS ', 'HD ', 'HON ', 'IBM ', 'INTC ', 'JNJ ', 'JPM ', 'KHC ', 'KO ', 'LIN ', 'LLY ', 'LMT ', 'LOW ', 'MA ', 'MCD ', 'MDLZ ', 'MDT ', 'MET ', 'META ', 'MMM ', 'MO ', 'MRK ', 'MS ', 'MSFT ', 'NEE ', 'NFLX ', 'NKE ', 'NVDA ', 'ORCL ', 'PEP ', 'PFE ', 'PG ', 'PM ', 'PYPL ', 'QCOM ', 'RTX ', 'SBUX ', 'SCHW ', 'SO ', 'SPG ', 'T ', 'TGT ', 'TMO ', 'TMUS ', 'TSLA ', 'TXN ', 'UNH ', 'UNP ', 'UPS ', 'USB ', 'V ', 'VZ ', 'WFC ', 'WMT ', 'XOM ']

In [6]:
# DataFrames for transactions, current portfolio, and outstanding investment
transactions = pd.DataFrame(columns=[
    'Date', 
    'Ticker',
    'Action',
    'Buy_or_Sell_Price',
    'Number_of_Shares',
    'Cash_Flow',
    'Entry_ID'
])

portfolio = pd.DataFrame(columns=[
    'Entry_ID',
    'Ticker',
    'Shares',
    'Purchase_Price',
    'Book_Value'
])

outstanding_investment = pd.DataFrame(columns=[
    'Date',
    'Outstanding_Invested_Amount'
])

# Also initialize DataFrames for an alternate portfolio consisting only of investing in the S&P500
alt_transactions = pd.DataFrame(columns=[
    'Date', 
    'Ticker',
    'Action',
    'Buy_or_Sell_Price',
    'Number_of_Shares',
    'Cash_Flow',
    'Entry_ID'
])

alt_portfolio = pd.DataFrame(columns=[
    'Entry_ID',
    'Ticker',
    'Shares',
    'Purchase_Price',
    'Book_Value'
])

In [7]:
# Function to simulate buying
def buy_stock(transactions, portfolio, date, ticker, close_price, entry_id=False):
    '''
    Simulates a stock purchase within an investment strategy. Updates transaction records, 
    the investment portfolio, and outstanding investment calculations.

    Args:
        transactions (pandas.DataFrame): A DataFrame to store historical transactions.
        portfolio (pandas.DataFrame): A DataFrame representing the current investment holdings.
        date (datetime.date): The date of the stock purchase.
        ticker (str): The ticker symbol of the purchased stock.
        close_price (float): The closing price of the stock at the time of purchase.
        entry_id (str, optional): A unique Entry_ID to identify the purchase. If not 
                                   provided, a new Entry_ID will be automatically generated.

    Returns:
        tuple: A tuple containing: 
               * updated transactions DataFrame
               * updated portfolio DataFrame
               * the Entry_ID used for the transaction (either provided or generated)
    '''

    shares_bought = investment_amount_per_buy_signal / close_price

    # if an entry_id is not already provided, generate a unique Entry_ID using a high-resolution timestamp
    if not entry_id:
        entry_id = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

    # Update transactions DataFrame
    new_transaction_entry = pd.DataFrame({
        'Date': [date],
        'Ticker': [ticker],
        'Action': ['Buy'],
        'Buy_or_Sell_Price': [close_price],
        'Number_of_Shares': [shares_bought],
        'Cash_Flow': [-investment_amount_per_buy_signal],
        'Entry_ID': [entry_id]
    })

    transactions = pd.concat([transactions, new_transaction_entry], ignore_index=True)

    # Update portfolio DataFrame
    new_portfolio_entry = pd.DataFrame({
        'Entry_ID': [entry_id],
        'Ticker': [ticker], 
        'Shares': [shares_bought], 
        'Purchase_Price': [close_price], 
        'Book_Value': [shares_bought * close_price]
    })

    portfolio = pd.concat([portfolio, new_portfolio_entry], ignore_index=True)

    # Update the outstanding investment
    update_outstanding_investment(date, investment_amount_per_buy_signal)

    # Logging the buy
    if output_transactions:
        print(f"Buy: {date.strftime('%Y-%m-%d')} - {ticker}, Shares: {shares_bought:.4f}, Price: {close_price}, Value: {investment_amount_per_buy_signal:.2f}")

    return transactions, portfolio, entry_id


# Function to simulate selling
def sell_stock(transactions, portfolio, date, entry_id, sell_price):
    '''
    Simulates a stock sale within an investment strategy. Updates transaction records, 
    the investment portfolio, and outstanding investment calculations.

    Args:
        transactions (pandas.DataFrame): A DataFrame to store historical transactions.
        portfolio (pandas.DataFrame): A DataFrame representing the current investment holdings.
        date (datetime.date): The date of the stock sale.
        entry_id (str): The unique Entry_ID identifying the stock position to sell.
        sell_price (float): The selling price of the stock.

    Returns:
        tuple: A tuple containing the updated transactions and portfolio DataFrames.
    '''

    # Locate the transaction to be sold using Entry_ID
    transaction_index = portfolio[portfolio['Entry_ID'] == entry_id].index
    if not transaction_index.empty:
        index = transaction_index[0]
        ticker = portfolio.at[index, 'Ticker']
        shares_sold = portfolio.at[index, 'Shares']
        sold_value = shares_sold * sell_price

        # Update transactions DataFrame with the sale information
        new_sale_entry = pd.DataFrame({
            'Date': [date],
            'Ticker': [ticker],
            'Action': ['Sell'],
            'Buy_or_Sell_Price': [sell_price],
            'Number_of_Shares': [shares_sold],
            'Cash_Flow': [sold_value],
            'Entry_ID': [entry_id]
        })
        
        transactions = pd.concat([transactions, new_sale_entry], ignore_index=True)

        # Update outstanding investment
        update_outstanding_investment(date, -investment_amount_per_buy_signal)

        # Remove the sold stock from the portfolio
        portfolio.drop(index, inplace=True)
        portfolio.reset_index(drop=True, inplace=True)

        # Logging the sale
        if output_transactions:
            print(f"Sell: {date.strftime('%Y-%m-%d')} - {ticker}, Shares: {shares_sold:.4f}, Price: {sell_price}, Value: {sold_value:.2f}")
    else:
        raise KeyError(f"ERROR: Entry ID {entry_id} not found in the portfolio.")
        
    return transactions, portfolio

# Update outstanding investment
def update_outstanding_investment(date, amount):
    '''
    Updates the outstanding investment record based on a transaction. 
    Carries over previous investment amounts or initializes new values as needed.

    Args:
        date (datetime.date): The date of the transaction affecting the outstanding investment.
        amount (float): The amount by which the outstanding investment is being changed 
                     (positive for investments, negative for returns). 
    '''

    if outstanding_investment.empty or date not in outstanding_investment['Date'].values:
        if not outstanding_investment.empty:
            # If there's a previous entry, carry over the last outstanding amount and add the current amount
            last_outstanding_amount = outstanding_investment.iloc[-1]['Outstanding_Invested_Amount']
            new_outstanding_amount = last_outstanding_amount + amount
        else:
            # If there are no previous entries, this is the first transaction
            new_outstanding_amount = amount
        outstanding_investment.loc[len(outstanding_investment)] = [date, new_outstanding_amount]
    else:
        # Update the existing entry for the date
        last_index = outstanding_investment[outstanding_investment['Date'] == date].index[-1]
        outstanding_investment.at[last_index, 'Outstanding_Invested_Amount'] += amount


In [8]:
# Source: https://stackoverflow.com/questions/8919718/financial-python-library-that-has-xirr-and-xnpv-function
def xnpv(rate, values, dates):
   '''
   Equivalent of Excel's XNPV function.
   '''
   return sum([v / (1 + rate)**((d - dates[0]).days / 365.0) for d, v in zip(dates, values)])

def xirr(transactions):
    """
    Calculates the XIRR (Internal Rate of Return with irregular dates) from a DataFrame of transactions.

    Args:
        transactions (pandas.DataFrame): A DataFrame with the columns:
            * Date (datetime): The date of the transaction.
            * Cash_Flow (float): The amount of cash invested or returned.

    Returns:
        float: The XIRR (Internal Rate of Return) as a percentage.
    """

    dates = transactions['Date'].dt.to_pydatetime()
    cash_flows = transactions['Cash_Flow'].to_numpy()

    try:
        return optimize.newton(lambda r: xnpv(r, cash_flows, dates), 0.0) * 100
    except RuntimeError:  # Handle cases where XIRR can't be calculated
        return optimize.brentq(lambda r: xnpv(r, cash_flows, dates), -1.0, 1e10) * 100

In [9]:
# Load historical data for each ticker
historical_data = {ticker: yf.download(ticker, start=start_date, end=end_date, progress=False) for ticker in sp100_stocks}
for ticker, data in historical_data.items():
    # The choice of 252 days is based on the typical number of trading days in a year for the U.S. stock market, which is commonly used as an approximation for a 52-week period.
    # The calculated 52-week high at any given row in the DataFrame represents the highest closing price over the preceding 252 trading days from that row
    data['52_Week_High'] = data['Close'].rolling(window=252).max()
    
    
# Also load historical data for SPY
spy_historical_data = yf.download('SPY', start=start_date, end=end_date, progress=False)

# Calculate the 52-Week_High for SPY
spy_historical_data['52_Week_High'] = spy_historical_data['Close'].rolling(window=252).max() 

In [10]:
# Simulation loop
current_date = datetime.strptime(start_date, '%Y-%m-%d')
end_simulation_date = datetime.strptime(end_date, '%Y-%m-%d')

while current_date <= end_simulation_date:
    day_has_transactions = False

    # For each day in the simulation, check if we should buy or sell each stock
    for ticker, data in historical_data.items():
        if current_date.strftime('%Y-%m-%d') in data.index:
            row = data.loc[current_date.strftime('%Y-%m-%d')]

            # Buy logic
            if not (ticker in portfolio['Ticker'].values and not allow_buy_stock_already_in_portfolio):
                # Skip buying if the stock is already in the portfolio and re-buying is not allowed
                if row['Close'] <= row['52_Week_High'] * (1 - buy_after_52_week_high_percentage_drop):
                    
                    # Buy stock according to the investment strategy
                    transactions, portfolio, this_entry_id = buy_stock(
                        transactions, portfolio, current_date, ticker, row['Close']
                    )
                    
                    # Also buy an equivalent amount in SPY in the alternative portfolio
                    spy_close_price = spy_historical_data.loc[current_date.strftime('%Y-%m-%d'), 'Close']
                    alt_transactions, alt_portfolio, _ = buy_stock(
                        alt_transactions, alt_portfolio, current_date, 'SPY', spy_close_price, this_entry_id
                    )
                    
                    day_has_transactions = True

            # Sell logic
            to_sell = []
            for index, stock in portfolio[portfolio['Ticker'] == ticker].iterrows():
                if row['Close'] >= stock['Purchase_Price'] * (1 + sell_after_percentage_rise_from_purchase_price):
                    # Add an entry to the DataFrame to_sell containing the stock to be sold. Uniquely
                    # identify the stock from the portfolio using Entry_ID
                    to_sell.append((stock['Entry_ID'], row['Close']))
            
            # Sell all stocks that meet the sell criteria
            for entry_id, sell_price in to_sell:
                
                # Sell stock according to the investment strategy
                transactions, portfolio = sell_stock(
                    transactions, portfolio, current_date, entry_id, sell_price
                )
                
                # Also sell corresponding stock purchases from SPY from the alternative portfolio
                spy_sell_price = spy_historical_data.loc[current_date.strftime('%Y-%m-%d'), 'Close']
                alt_transactions, alt_portfolio = sell_stock(
                    alt_transactions, alt_portfolio, current_date, entry_id, spy_sell_price
                )
                
                day_has_transactions = True


    if output_portfolio:
        # Print the current portfolio:
        portfolio_without_entry_id = portfolio.drop('Entry_ID', axis=1)
        print(f"\nCurrent Portfolio as of {current_date.strftime('%Y-%m-%d')}:")
        print(portfolio_without_entry_id)
        print("\n")
    
    # If no transactions occurred on this day, carry over the last outstanding investment amount
    if not day_has_transactions:
        update_outstanding_investment(current_date, 0)

    if output_outstanding_investment:
        # Print statements for current date and outstanding investment
        if not outstanding_investment.empty:
            current_outstanding_investment = outstanding_investment[outstanding_investment['Date'] == current_date.strftime('%Y-%m-%d')]['Outstanding_Invested_Amount'].iloc[-1]
            print("Date:", current_date.strftime('%Y-%m-%d'), "Outstanding Investment: $ ", "{:,.2f}".format(current_outstanding_investment))
        else:
            print("Date:", current_date.strftime('%Y-%m-%d'), "Outstanding Investment: $ 0.00")


    current_date += timedelta(days=buy_sell_interval_in_days)

Buy: 2020-04-02 - AAPL, Shares: 16.3312, Price: 61.23249816894531, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - ABBV , Shares: 13.3103, Price: 75.12999725341797, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - ACN , Shares: 6.3992, Price: 156.27000427246094, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - ADBE , Shares: 3.2899, Price: 303.9599914550781, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - AIG , Shares: 47.0588, Price: 21.25, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - AMD , Shares: 22.4770, Price: 44.4900016784668, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - AVGO , Shares: 4.2183,

Buy: 2020-04-02 - TSLA , Shares: 33.0055, Price: 30.29800033569336, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - TXN , Shares: 9.8020, Price: 102.0199966430664, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - UNH , Shares: 4.1590, Price: 240.44000244140625, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - UNP , Shares: 7.0522, Price: 141.8000030517578, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - UPS , Shares: 10.7481, Price: 93.04000091552734, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - USB , Shares: 30.6091, Price: 32.66999816894531, Value: 1000.00
Buy: 2020-04-02 - SPY, Shares: 3.9709, Price: 251.8300018310547, Value: 1000.00
Buy: 2020-04-02 - V , Shares:

Buy: 2020-04-09 - SPY, Shares: 3.5945, Price: 278.20001220703125, Value: 1000.00
Buy: 2020-04-09 - MS , Shares: 24.3427, Price: 41.08000183105469, Value: 1000.00
Buy: 2020-04-09 - SPY, Shares: 3.5945, Price: 278.20001220703125, Value: 1000.00
Sell: 2020-04-09 - NEE , Shares: 18.0473, Price: 61.31999969482422, Value: 1106.66
Sell: 2020-04-09 - SPY, Shares: 4.0292, Price: 278.20001220703125, Value: 1120.92
Buy: 2020-04-09 - RTX , Shares: 15.4536, Price: 64.70999908447266, Value: 1000.00
Buy: 2020-04-09 - SPY, Shares: 3.5945, Price: 278.20001220703125, Value: 1000.00
Buy: 2020-04-09 - SBUX , Shares: 13.5355, Price: 73.87999725341797, Value: 1000.00
Buy: 2020-04-09 - SPY, Shares: 3.5945, Price: 278.20001220703125, Value: 1000.00
Sell: 2020-04-09 - SPG , Shares: 16.4258, Price: 68.16999816894531, Value: 1119.74
Sell: 2020-04-09 - SPY, Shares: 3.7717, Price: 278.20001220703125, Value: 1049.30
Sell: 2020-04-09 - USB , Shares: 30.6091, Price: 38.04999923706055, Value: 1164.68
Sell: 2020-04-09 

Sell: 2020-04-28 - XOM , Shares: 24.7525, Price: 44.970001220703125, Value: 1113.12
Sell: 2020-04-28 - SPY, Shares: 3.9709, Price: 285.7300109863281, Value: 1134.61
Buy: 2020-04-29 - COF , Shares: 14.3369, Price: 69.75, Value: 1000.00
Buy: 2020-04-29 - SPY, Shares: 3.4105, Price: 293.2099914550781, Value: 1000.00
Buy: 2020-04-29 - COP , Shares: 23.6911, Price: 42.209999084472656, Value: 1000.00
Buy: 2020-04-29 - SPY, Shares: 3.4105, Price: 293.2099914550781, Value: 1000.00
Sell: 2020-04-29 - CVX , Shares: 11.8610, Price: 94.62000274658203, Value: 1122.29
Sell: 2020-04-29 - SPY, Shares: 3.5945, Price: 293.2099914550781, Value: 1053.95
Sell: 2020-04-29 - DIS , Shares: 10.3125, Price: 112.25, Value: 1157.57
Sell: 2020-04-29 - SPY, Shares: 3.9709, Price: 293.2099914550781, Value: 1164.32
Sell: 2020-04-29 - FDX , Shares: 8.5734, Price: 129.47999572753906, Value: 1110.08
Sell: 2020-04-29 - SPY, Shares: 3.9709, Price: 293.2099914550781, Value: 1164.32
Sell: 2020-04-29 - GOOG , Shares: 16.4428

Buy: 2020-05-28 - C , Shares: 20.3417, Price: 49.15999984741211, Value: 1000.00
Buy: 2020-05-28 - SPY, Shares: 3.3007, Price: 302.9700012207031, Value: 1000.00
Buy: 2020-05-28 - F , Shares: 170.9402, Price: 5.849999904632568, Value: 1000.00
Buy: 2020-05-28 - SPY, Shares: 3.3007, Price: 302.9700012207031, Value: 1000.00
Sell: 2020-05-28 - LIN , Shares: 5.5935, Price: 200.0399932861328, Value: 1118.92
Sell: 2020-05-28 - SPY, Shares: 3.5511, Price: 302.9700012207031, Value: 1075.89
Buy: 2020-05-28 - MS , Shares: 22.5225, Price: 44.400001525878906, Value: 1000.00
Buy: 2020-05-28 - SPY, Shares: 3.3007, Price: 302.9700012207031, Value: 1000.00
Sell: 2020-05-28 - NEE , Shares: 17.9646, Price: 62.84000015258789, Value: 1128.90
Sell: 2020-05-28 - SPY, Shares: 3.5180, Price: 302.9700012207031, Value: 1065.86
Buy: 2020-05-28 - T , Shares: 42.6272, Price: 23.45921516418457, Value: 1000.00
Buy: 2020-05-28 - SPY, Shares: 3.3007, Price: 302.9700012207031, Value: 1000.00
Buy: 2020-05-29 - GS , Shares:

...

Sell: 2022-04-06 - AMT , Shares: 4.1220, Price: 268.6199951171875, Value: 1107.25
Sell: 2022-04-06 - SPY, Shares: 2.3074, Price: 446.5199890136719, Value: 1030.32
Sell: 2022-04-06 - PM , Shares: 11.2524, Price: 99.0199966430664, Value: 1114.21
Sell: 2022-04-06 - SPY, Shares: 2.3806, Price: 446.5199890136719, Value: 1062.97
Buy: 2022-04-07 - BAC , Shares: 25.3872, Price: 39.38999938964844, Value: 1000.00
Buy: 2022-04-07 - SPY, Shares: 2.2283, Price: 448.7699890136719, Value: 1000.00
Sell: 2022-04-08 - TGT , Shares: 4.7234, Price: 233.33999633789062, Value: 1102.17
Sell: 2022-04-08 - SPY, Shares: 2.3074, Price: 447.57000732421875, Value: 1032.74
Buy: 2022-04-11 - TSLA , Shares: 3.0740, Price: 325.30999755859375, Value: 1000.00
Buy: 2022-04-11 - SPY, Shares: 2.2731, Price: 439.9200134277344, Value: 1000.00
Buy: 2022-04-14 - WFC , Shares: 21.5750, Price: 46.349998474121094, Value: 1000.00
Buy: 2022-04-14 - SPY, Shares: 2.2842, Price: 437.7900085449219, Value: 1000.00
Buy: 2022-04-18 - CSCO

Buy: 2022-06-16 - AIG , Shares: 20.0120, Price: 49.970001220703125, Value: 1000.00
Buy: 2022-06-16 - SPY, Shares: 2.7274, Price: 366.6499938964844, Value: 1000.00
Buy: 2022-06-16 - DOW , Shares: 18.0440, Price: 55.41999816894531, Value: 1000.00
Buy: 2022-06-16 - SPY, Shares: 2.7274, Price: 366.6499938964844, Value: 1000.00
Buy: 2022-06-16 - MA , Shares: 3.2386, Price: 308.7799987792969, Value: 1000.00
Buy: 2022-06-16 - SPY, Shares: 2.7274, Price: 366.6499938964844, Value: 1000.00
Buy: 2022-06-17 - COP , Shares: 10.6678, Price: 93.73999786376953, Value: 1000.00
Buy: 2022-06-17 - SPY, Shares: 2.7333, Price: 365.8599853515625, Value: 1000.00
Buy: 2022-06-17 - CVS , Shares: 11.2790, Price: 88.66000366210938, Value: 1000.00
Buy: 2022-06-17 - SPY, Shares: 2.7333, Price: 365.8599853515625, Value: 1000.00
Buy: 2022-06-22 - CAT , Shares: 5.3240, Price: 187.8300018310547, Value: 1000.00
Buy: 2022-06-22 - SPY, Shares: 2.6710, Price: 374.3900146484375, Value: 1000.00
Buy: 2022-06-22 - MO , Shares:

Buy: 2022-09-26 - TXN , Shares: 6.2321, Price: 160.4600067138672, Value: 1000.00
Buy: 2022-09-26 - SPY, Shares: 2.7449, Price: 364.30999755859375, Value: 1000.00
Buy: 2022-09-27 - EXC , Shares: 24.9066, Price: 40.150001525878906, Value: 1000.00
Buy: 2022-09-27 - SPY, Shares: 2.7519, Price: 363.3800048828125, Value: 1000.00
Buy: 2022-09-27 - PM , Shares: 11.4916, Price: 87.0199966430664, Value: 1000.00
Buy: 2022-09-27 - SPY, Shares: 2.7519, Price: 363.3800048828125, Value: 1000.00
Buy: 2022-09-29 - AAPL, Shares: 7.0185, Price: 142.47999572753906, Value: 1000.00
Buy: 2022-09-29 - SPY, Shares: 2.7564, Price: 362.7900085449219, Value: 1000.00
Buy: 2022-09-29 - PG , Shares: 7.7700, Price: 128.6999969482422, Value: 1000.00
Buy: 2022-09-29 - SPY, Shares: 2.7564, Price: 362.7900085449219, Value: 1000.00
Sell: 2022-10-06 - CVX , Shares: 6.9075, Price: 161.4199981689453, Value: 1115.01
Sell: 2022-10-06 - SPY, Shares: 2.7178, Price: 373.20001220703125, Value: 1014.27
Buy: 2022-10-06 - DUK , Share

Sell: 2023-03-31 - ORCL , Shares: 12.0890, Price: 92.91999816894531, Value: 1123.31
Sell: 2023-03-31 - SPY, Shares: 2.2384, Price: 409.3900146484375, Value: 916.37
Sell: 2023-03-31 - SO , Shares: 15.8579, Price: 69.58000183105469, Value: 1103.39
Sell: 2023-03-31 - SPY, Shares: 2.5236, Price: 409.3900146484375, Value: 1033.13
Buy: 2023-04-06 - CAT , Shares: 4.7808, Price: 209.1699981689453, Value: 1000.00
Buy: 2023-04-06 - SPY, Shares: 2.4439, Price: 409.19000244140625, Value: 1000.00
Sell: 2023-04-14 - GS , Shares: 3.2945, Price: 336.9200134277344, Value: 1109.97
Sell: 2023-04-14 - SPY, Shares: 2.5642, Price: 412.4599914550781, Value: 1057.62
Sell: 2023-04-26 - SBUX , Shares: 9.9880, Price: 111.47000122070312, Value: 1113.36
Sell: 2023-04-26 - SPY, Shares: 2.1518, Price: 404.3599853515625, Value: 870.12
Sell: 2023-04-27 - MSFT , Shares: 3.6492, Price: 304.8299865722656, Value: 1112.40
Sell: 2023-04-27 - SPY, Shares: 2.3472, Price: 412.4100036621094, Value: 968.01
Sell: 2023-05-17 - NVD

Sell: 2023-12-04 - LOW , Shares: 5.3694, Price: 207.5500030517578, Value: 1114.42
Sell: 2023-12-04 - SPY, Shares: 2.3606, Price: 456.69000244140625, Value: 1078.04
Sell: 2023-12-12 - KHC , Shares: 29.6209, Price: 37.2599983215332, Value: 1103.67
Sell: 2023-12-12 - SPY, Shares: 2.2579, Price: 464.1000061035156, Value: 1047.89
Buy: 2023-12-12 - ORCL , Shares: 9.9197, Price: 100.80999755859375, Value: 1000.00
Buy: 2023-12-12 - SPY, Shares: 2.1547, Price: 464.1000061035156, Value: 1000.00
Sell: 2023-12-14 - FDX , Shares: 3.9872, Price: 279.4599914550781, Value: 1114.27
Sell: 2023-12-14 - SPY, Shares: 2.3039, Price: 472.010009765625, Value: 1087.48
Sell: 2023-12-14 - HD , Shares: 3.1629, Price: 351.80999755859375, Value: 1112.72
Sell: 2023-12-14 - SPY, Shares: 2.3279, Price: 472.010009765625, Value: 1098.80
Sell: 2023-12-14 - MET , Shares: 16.5948, Price: 66.9000015258789, Value: 1110.19
Sell: 2023-12-14 - SPY, Shares: 2.3720, Price: 472.010009765625, Value: 1119.59
Sell: 2023-12-14 - MS , 

In [11]:
# Convert end_date to datetime for comparison
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

# End of simulation: Sell all outstanding stocks
for index, stock in portfolio.iterrows():
    print("\n--- End of Simulation Sell-Off ---")
    ticker = stock['Ticker']
    entry_id = stock['Entry_ID']

    # Check if we have historical data for this ticker
    if ticker in historical_data:
        # Get the data for this ticker
        data = historical_data[ticker]

        # Find the last available price before or on the end_date
        # Filter the data up to and including the end_date
        filtered_data = data[data.index <= end_date_dt]

        if not filtered_data.empty:
            # Get the last available closing price
            last_price = filtered_data['Close'].iloc[-1]
            
            # Sell the stock
            transactions, portfolio = sell_stock(
                transactions, portfolio, end_date_dt, entry_id, last_price
            )
            
            # Also sell SPY from the alternative portfolio
            spy_filtered_data = spy_historical_data[spy_historical_data.index <= end_date_dt]
            spy_last_price = spy_filtered_data['Close'].iloc[-1]
            alt_transactions, alt_portfolio = sell_stock(
                alt_transactions, alt_portfolio, end_date_dt, entry_id, spy_last_price
            )
            
        else:
            raise KeyError(f"No available closing price for {ticker} on or before {end_date}")
    else:
        raise KeyError(f"No historical data found for {ticker}")

# Print outstanding investment for the last day, which should be $0 after all stock has been sold
end_simulation_date = datetime.strptime(end_date, '%Y-%m-%d')
if not outstanding_investment.empty:
    ending_outstanding_investment = outstanding_investment.iloc[-1]['Outstanding_Invested_Amount']
    print("Ending Date:", end_date, "Ending Outstanding Investment: $", "{:,.2f}".format(ending_outstanding_investment))
else:
    print("Ending Date:", end_date, "Ending Outstanding Investment: $0.00")


--- End of Simulation Sell-Off ---
Sell: 2024-04-02 - T , Shares: 42.6272, Price: 17.5, Value: 745.98
Sell: 2024-04-02 - SPY, Shares: 3.3007, Price: 522.1599731445312, Value: 1723.47

--- End of Simulation Sell-Off ---
Sell: 2024-04-02 - INTC , Shares: 18.6498, Price: 44.52000045776367, Value: 830.29
Sell: 2024-04-02 - SPY, Shares: 2.4666, Price: 522.1599731445312, Value: 1287.98

--- End of Simulation Sell-Off ---
Sell: 2024-04-02 - PYPL , Shares: 4.1117, Price: 65.02999877929688, Value: 267.38
Sell: 2024-04-02 - SPY, Shares: 2.2046, Price: 522.1599731445312, Value: 1151.17

--- End of Simulation Sell-Off ---
Sell: 2024-04-02 - DIS , Shares: 6.2645, Price: 121.52999877929688, Value: 761.32
Sell: 2024-04-02 - SPY, Shares: 2.1401, Price: 522.1599731445312, Value: 1117.47

--- End of Simulation Sell-Off ---
Sell: 2024-04-02 - C , Shares: 15.6986, Price: 63.459999084472656, Value: 996.23
Sell: 2024-04-02 - SPY, Shares: 2.1951, Price: 522.1599731445312, Value: 1146.19

--- End of Simulati

In [12]:
# Calculate total investment and returns
total_invested = transactions[transactions['Action'] == 'Buy']['Cash_Flow'].sum()
total_returned = transactions[transactions['Action'] == 'Sell']['Cash_Flow'].sum()
total_gain_loss = total_returned + total_invested  # Since invested amount is negative

print("For custom investment strategy:")
print("Total Cumulative Invested: $", "{:,.2f}".format(-total_invested))
print("Total Cumulative Returned: $", "{:,.2f}".format(total_returned))
print("Total Gain/Loss: $", "{:,.2f}".format(total_gain_loss))

strategy_xirr = xirr(transactions)
print("XIRR: {:.2f}%".format(strategy_xirr)) 

# Calculate total investment and returns for alternative S&P 500 strategy
alt_total_invested = alt_transactions[alt_transactions['Action'] == 'Buy']['Cash_Flow'].sum()
alt_total_returned = alt_transactions[alt_transactions['Action'] == 'Sell']['Cash_Flow'].sum()
alt_total_gain_loss = alt_total_returned + alt_total_invested  # Since invested amount is negative

print("\nFor S&P500 alternative investment strategy:")
print("Total Cumulative Invested: $", "{:,.2f}".format(-alt_total_invested))
print("Total Cumulative Returned: $", "{:,.2f}".format(alt_total_returned))
print("Total Gain/Loss: $", "{:,.2f}".format(alt_total_gain_loss))

alt_strategy_xirr = xirr(alt_transactions) 
print("XIRR: {:.2f}%".format(alt_strategy_xirr)) 

For custom investment strategy:
Total Cumulative Invested: $ 427,000.00
Total Cumulative Returned: $ 468,526.47
Total Gain/Loss: $ 41,526.47
XIRR: 58.79%

For S&P500 alternative investment strategy:
Total Cumulative Invested: $ 427,000.00
Total Cumulative Returned: $ 467,126.01
Total Gain/Loss: $ 40,126.01
XIRR: 39.10%


In [13]:
# Show the outstanding investment (how much cash you have used to acquire your current portfolio) per trading day
for index, row in outstanding_investment.iterrows():
    print("Date: ",row['Date'].strftime('%Y-%m-%d'), "Outstanding Investment: $ ", "{:,.2f}".format(row['Outstanding_Invested_Amount']))

Date:  2019-04-04 Outstanding Investment: $  0.00
Date:  2019-04-05 Outstanding Investment: $  0.00
Date:  2019-04-06 Outstanding Investment: $  0.00
Date:  2019-04-07 Outstanding Investment: $  0.00
Date:  2019-04-08 Outstanding Investment: $  0.00
Date:  2019-04-09 Outstanding Investment: $  0.00
Date:  2019-04-10 Outstanding Investment: $  0.00
Date:  2019-04-11 Outstanding Investment: $  0.00
Date:  2019-04-12 Outstanding Investment: $  0.00
Date:  2019-04-13 Outstanding Investment: $  0.00
Date:  2019-04-14 Outstanding Investment: $  0.00
Date:  2019-04-15 Outstanding Investment: $  0.00
Date:  2019-04-16 Outstanding Investment: $  0.00
Date:  2019-04-17 Outstanding Investment: $  0.00
Date:  2019-04-18 Outstanding Investment: $  0.00
Date:  2019-04-19 Outstanding Investment: $  0.00
Date:  2019-04-20 Outstanding Investment: $  0.00
Date:  2019-04-21 Outstanding Investment: $  0.00
Date:  2019-04-22 Outstanding Investment: $  0.00
Date:  2019-04-23 Outstanding Investment: $  0.00


Date:  2021-06-27 Outstanding Investment: $  10,000.00
Date:  2021-06-28 Outstanding Investment: $  10,000.00
Date:  2021-06-29 Outstanding Investment: $  10,000.00
Date:  2021-06-30 Outstanding Investment: $  10,000.00
Date:  2021-07-01 Outstanding Investment: $  10,000.00
Date:  2021-07-02 Outstanding Investment: $  10,000.00
Date:  2021-07-03 Outstanding Investment: $  10,000.00
Date:  2021-07-04 Outstanding Investment: $  10,000.00
Date:  2021-07-05 Outstanding Investment: $  10,000.00
Date:  2021-07-06 Outstanding Investment: $  10,000.00
Date:  2021-07-07 Outstanding Investment: $  10,000.00
Date:  2021-07-08 Outstanding Investment: $  10,000.00
Date:  2021-07-09 Outstanding Investment: $  10,000.00
Date:  2021-07-10 Outstanding Investment: $  10,000.00
Date:  2021-07-11 Outstanding Investment: $  10,000.00
Date:  2021-07-12 Outstanding Investment: $  10,000.00
Date:  2021-07-13 Outstanding Investment: $  10,000.00
Date:  2021-07-14 Outstanding Investment: $  10,000.00
Date:  202